# The Battle of Neighborhoods


## Find the best locatoion for cofee shop

<br>

## Introduction

Our client want to open new cofee shop in Riyadh city and he is asking about the best location for his business.
For sure he can change anything after he open the cofee shop but not the location, so we have to help him choose the right location, which is the one with lowest number of cofee shops for less cometitions.

<br>


## Business Challenge


I will use Foursquare database to provide our client the best location by clustering the areas by its distance from the center fo the city and find the area with lowset number of cofee shops.
<br>
<br>


## Data requirements

For this assignment, I will be utilizing the Foursquare API to pull the following location data in Riyadh

- Venue Name
- Venue ID
- Venue Location
- Venue Category
- venue distance
<br>

I will cluster the areas acording to distance from the center and number of venues category equal to **Cofee Shop**

<br>

In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 1. Get Riyadh's latitude and longitude values

world cities coordinates already available on simplemaps.com website..
I've uploaded the world cities coordinates file to github  

In [4]:
cities_df = pd.read_csv('https://raw.githubusercontent.com/erefaie/Coursera_Capstone/master/sa.csv')

cities_df.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Riyadh,24.653837,46.715683,Saudi Arabia,SA,Ar Riyāḑ,primary,4465000.0,4205961.0
1,Jeddah,21.542381,39.197969,Saudi Arabia,SA,Makkah al Mukarramah,NaN,3012000.0,2867446.0
2,Ad Dammām,26.434417,50.103263,Saudi Arabia,SA,Ash Sharqīyah,admin,2054710.0,768602.0
3,Medina,24.468611,39.614167,Saudi Arabia,SA,Al Madīnah al Munawwarah,admin,1010000.0,1010000.0
4,Aţ Ţā’if,21.270278,40.415833,Saudi Arabia,SA,Makkah al Mukarramah,NaN,657282.0,530848.0


Find Riyadh latitude & longitude in the dataframe

In [5]:
RY_cordinates = cities_df.loc[cities_df['city'] == 'Riyadh']
RY_latitude = RY_cordinates.get_value(0, 'lat')
RY_longitude = RY_cordinates.get_value(0, 'lng')


print(RY_latitude)
print(RY_longitude)

24.653837
46.715683


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


<br>
<br>



Here i am going to start utilizing the Foursquare API to explore the neighborhoods in Riyadh and segment them.

### 2. Define Foursquare Credentials, Version, and build the needed URL

In [6]:
CLIENT_ID = '2LUYJPERSOIYVIPLYFOHOGV1ZPASRSOVIUGRQ53ABMNZ3KTU' # My Foursquare ID
CLIENT_SECRET = 'TH4VCMJBMD1ZLN0BSO3OVSABF50Z31VNCNDBZLHBZOBR42QA' # My Foursquare Secret
VERSION = '20180605' # Foursquare API version


First, I'll create the GET request URL. Name your URL **url**.

In [7]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}&radious={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
RY_latitude,
RY_longitude,
100,
500)

print(url)

https://api.foursquare.com/v2/venues/explore?&client_id=2LUYJPERSOIYVIPLYFOHOGV1ZPASRSOVIUGRQ53ABMNZ3KTU&client_secret=TH4VCMJBMD1ZLN0BSO3OVSABF50Z31VNCNDBZLHBZOBR42QA&v=20180605&ll=24.653837,46.715683&limit=100&radious=500



### 3. Send the GET request and examine the resutls


In [8]:
results = requests.get(url).json()

Now will clean the json and structure it into a pandas dataframe

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Read the resulted information from json file dsinto pandas dataframe.

In [10]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON


# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.distance']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]




Will use head function to have a look to the data

In [11]:
nearby_venues.head()


,name,categories,lat,lng,distance
0,dr.CAFE COFFEE,Coffee Shop,24.656452,46.715608,291
1,Thiqah CITC (مركز المؤتمرات وتقنية المعلومات -...,IT Services,24.656250,46.715990,270
2,Black harvest,Coffee Shop,24.656657,46.712936,419
3,dr.CAFE COFFEE | د. كيف,Coffee Shop,24.649550,46.715107,480
4,Starbucks,Coffee Shop,24.656467,46.713449,369


<br> <br>

## Exploratory Data Analysis

In [12]:
nearby_venues.shape

(100, 5)

<font size="5">Venues dataset comprise of 100 observations and 5 characteristics</font>

In [13]:
nearby_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
name          100 non-null object
categories    100 non-null object
lat           100 non-null float64
lng           100 non-null float64
distance      100 non-null int64
dtypes: float64(2), int64(1), object(2)
memory usage: 4.0+ KB
